In [2]:
import pickle as pkl
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, roc_auc_score

import sys
sys.path.insert(1, '../')
from encoding import prefix_bin
import numpy as np
import json
import itertools

In [3]:
prefix =1
classifier = 'xgb'
evaluation = 'fscore'
dataset_label = 'bpic17'
evaluation = 'aggregated'

training_rebalance = True

In [4]:
def trueTo1(x):
    y_true = []
    for i in x:
        if i =='True':
            y_true.append(1)
        else:
            y_true.append(0)
    return y_true

In [5]:
def return_no_0(y, evaluationlist):
    y_0 =[]
    eval_0=[]
    
    for pos, i in enumerate(evaluationlist):
        if i != 0.0:
            y_0.append(y[pos])
            eval_0.append(i)
            
    return y_0, eval_0

In [6]:
def variant_collection(events_in_window):
    window_variant = {}
    for t in events_in_window:
        values = [i for i in t.encoded if t.encoded[i]==1]
        activities = str(sorted([i for i in values if 'activity' in i]))

        if activities not in window_variant.keys():
            window_variant[activities] =1
        else:
            window_variant[activities] +=1
    return window_variant

In [7]:
def variant_coverage(training_data, test):
    test_variant = {}
    
    train_variant = variant_collection(training_data)
    for updates in test_data.keys():
        test_window_variant = variant_collection(test_data[updates])
        test_variant[updates] = test_window_variant
        
    appeared_in_train_list = []
    for i in test_variant.keys():
        appeared_in_train = 0
        for t in test_variant[i].keys():
            if t in train_variant.keys():
                appeared_in_train += test_variant[i][t]

        appeared_in_train_list.append(appeared_in_train)
    return appeared_in_train_list

In [11]:
evaluation_dict = {}
evaluation = 'aggregated'
dataset_label = 'bpic17'
training_rebalance = True
coverage_df = {}

for classifier in ['hatc', 'rf', 'xgb']:

    with open('../dataset_parameters.json','r') as json_file:
        parameters = json.load(json_file)[dataset_label]
        print(parameters)
        key_pair = parameters['key_pair']
        catatars = parameters['categorical_attrs']
        maximum_prefixs = parameters['maximum_prefix']

    for prefix in [2,3]:
        evaluation_dict[prefix] = {}
        train_variant = {}
        test_variant = {}


        training_data = '../result/%s/%s/Finished cases/train_rebalance_%s prefix_%s training window label retrained.pkl'%(dataset_label, classifier, training_rebalance, prefix)
        test_data = '../result/%s/%s/Finished cases/train_rebalance_%s prefix_%s test window label retrained.pkl'%(dataset_label, classifier, training_rebalance, prefix)
        result = '../result/%s/%s/Finished cases/train_rebalance_%s prefix_%s update label retrained.pkl'%(dataset_label, classifier, training_rebalance, prefix)

        with open(training_data, 'rb') as openfile:
            training_data = pkl.load(openfile)

        with open(test_data, 'rb') as openfile:
            test_data = pkl.load(openfile)

        with open(result, 'rb') as openfile:
            result = pkl.load(openfile)
        updated_train_index = list(training_data.keys())[-1]
        if classifier == 'hatc':            
            before_retrain_data = training_data[200][0]
            after_retrain_data = training_data[updated_train_index][0]

        else:
            before_retrain_data = training_data[200][0]
            after_retrain_data = training_data[updated_train_index][0]
            
        variant_cover_before = variant_coverage(training_data = before_retrain_data, test = test_data)
        variant_cover_after = variant_coverage(training_data = after_retrain_data, test = test_data)
        variant_cover = variant_cover_before[:updated_train_index-200]+variant_cover_after[updated_train_index-200:]

        evaluationlist = {'fscore':[], 'precision':[], 'recall':[]}


        for updates in result[prefix].keys():
            if classifier == 'hatc':
                y_pred = [max(i, key=i.get) for i in result[prefix][updates]['y_pred']]
                y_pred = trueTo1(y_pred)
            else:
                y_pred = [np.argmax(i[0]) for i in result[prefix][updates]['y_pred']]

            y_true = trueTo1(result[prefix][updates]['y_true'])

            evaluationlist['fscore'].append(f1_score(y_true = y_true, y_pred = y_pred, average='weighted'))
            evaluationlist['precision'].append(precision_score(y_true = y_true, y_pred = y_pred, average='weighted'))
            evaluationlist['recall'].append(recall_score(y_true = y_true, y_pred = y_pred, average='weighted'))

        y = [i/30 for i in variant_cover]
        coverage_df[prefix] = y
        for i in evaluationlist:
            y_, eval_ = return_no_0(y, evaluationlist[i])
            y_delta = [y_[pos]-y[pos-1] for pos in range(len(y_)) if pos !=0]
            eval_delta = [eval_[pos]-eval_[pos-1] for pos in range(len(eval_)) if pos !=0]
            evaluation_dict[prefix][i] = np.corrcoef(y_, eval_)[0][1] 
    coverage_df = pd.DataFrame.from_dict(coverage_df)
    coverage_df_dir = '../result/%s/%s/rebalanced_%s variant label retrained.csv'%(dataset_label, classifier, str(training_rebalance))
    coverage_df.to_csv(coverage_df_dir, index=False)
    
    eval_df = pd.DataFrame.from_dict(evaluation_dict)
    eval_df_dir = '../result/%s/%s/rebalanced_%s variant corr label retrained.pkl'%(dataset_label, classifier, str(training_rebalance))
    with open(eval_df_dir, 'wb') as r:
        pkl.dump(eval_df, r)

{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Resource': 'resource', 'Start Timestamp': 'ts'}, 'categorical_attrs': ['activity', 'resource'], 'maximum_prefix': 15}


c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 

{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Resource': 'resource', 'Start Timestamp': 'ts'}, 'categorical_attrs': ['activity', 'resource'], 'maximum_prefix': 15}


c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true 

{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Resource': 'resource', 'Start Timestamp': 'ts'}, 'categorical_attrs': ['activity', 'resource'], 'maximum_prefix': 15}


c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true 